1. 将"*-pred.json"文件进行后处理
2. 将submit文件压缩打包

In [4]:
from glob import glob
from tqdm import tqdm
import os
import json
import cv2
import numpy as np
from pathlib import Path
import shutil
# from prettyprinter import cpprint, set_default_style


In [5]:
def check_table_format(table):
    if 'layout' not in table or 'cells' not in table:
        return False
    
    for cell in table['cells']:
        if 'col_start_idx' not in cell or 'col_end_idx' not in cell or 'row_start_idx' not in cell or 'row_end_idx' not in cell:
            return False
        if 'bbox' not in cell or 'segmentation' not in cell:
            return False
        if 'transcript' not in cell:
            return False

    return True



dataset = "test_A_jpg480max"
pred_json_dir = f"./output/structure_result/{dataset}-pred.json"
jpg_json_path = f"/media/ubuntu/Date12/TableStruct/new_data/{dataset}.json"
jpg_info = json.load(open(jpg_json_path, 'r'))

submit_dir = "./output/structure_result/submit"
if os.path.exists(submit_dir):
    shutil.rmtree(submit_dir)
Path(submit_dir).mkdir(parents=True, exist_ok=True)

for pred_json_path in tqdm(glob(os.path.join(pred_json_dir, "*-pred.json"))):
    pred_json_name = os.path.basename(pred_json_path)
    save_json_path = os.path.join(submit_dir, pred_json_name)

    table = json.load(open(pred_json_path, "r"))
    table = post_process(table)

    img_id = pred_json_name.split(".")[0].replace("-pred", "")
    for idx, cell in enumerate(table['cells']):
        cell["bbox"] = np.array(cell["bbox"]) / jpg_info[img_id]["scale"]
        cell["bbox"] = cell["bbox"].astype(np.int32).tolist()
        segmentation = []
        for seg in cell["segmentation"]:
            seg = np.array(seg) / jpg_info[img_id]["scale"]
            seg = seg.astype(np.int32).tolist()
            segmentation.append(seg)
        cell["segmentation"] =segmentation 
        table['cells'][idx] = cell

    if check_table_format(table) == False:
        print(f"check_table_format error: {pred_json_path}")
        break
    json.dump(table, open(save_json_path, "w"), indent=4, ensure_ascii=False)

# Q.用这个好像有问题？ A.少了一层submit.zip/submit
# shutil.make_archive(submit_dir, 'zip', submit_dir)

  0%|          | 14/5187 [00:00<00:37, 137.87it/s]

100%|██████████| 5187/5187 [00:32<00:00, 158.11it/s]
